# 전처리 핵심 요약: 이론과 실습

이 노트북은 텍스트 전처리의 핵심 개념과 최소 실행 예제를 한 곳에 정리해, 학습자가 바로 실행하며 복습할 수 있도록 구성했습니다.

- 목적: 개념 고착, 실행 중심 복습, 이후 Subword 섹션 추가 시 기반 정리
- 데이터: 간단한 예제 텍스트 중심 (대용량 파일은 미사용)
- 실행 방식: 각 섹션의 코드 셀을 순서대로 실행



## 📚 목차
1. 핵심 키워드 한눈에 보기
2. 전처리 파이프라인 개요
3. 텍스트 정규화와 토큰화
4. 불용어 제거
5. 정규표현식 핵심 패턴
6. 정수 인코딩(Integer Encoding)
7. 시퀀스 패딩(Padding)
8. 원-핫 인코딩(One-Hot Encoding)
9. 워드클라우드(Word Cloud)
10. 체크리스트 & 실전 팁
11. 복습 퀴즈 & 미니 과제



## 1) 핵심 키워드 한눈에 보기
- **Token**: 텍스트의 기본 단위
- **Normalization**: 대소문자 통일, 공백/특수문자 처리 등 규칙화
- **Stopwords**: 분석에 덜 유의미한 단어들(예: the, and, 은/는/이/가)
- **Stemming vs Lemmatization**: 어간 추출(형태만) vs 표제어 추출(사전기반 의미)
- **Regex(정규표현식)**: 패턴 기반 치환/추출
- **Integer Encoding**: 토큰을 정수로 매핑
- **Padding**: 시퀀스 길이 표준화
- **One-Hot**: 정수 레이블/토큰을 진리표 형태의 벡터로 표현
- **OOV(Out-Of-Vocabulary)**: 학습 사전에 없는 단어 처리 전략



## 2) 전처리 파이프라인 개요
1. 텍스트 정규화(소문자/공백/특수문자/숫자 처리)
2. 토큰화
3. 불용어 제거(도메인 맞춤 확장)
4. 표제어/어간 처리(언어/도메인/모델 특성에 맞춰 선택)
5. 정수 인코딩 → 패딩
6. 필요 시 원-핫/임베딩 선택



In [ ]:
# 공통 예제 텍스트와 유틸 함수
sample_text = "The Matrix is everywhere! It's all around us; here, even in this room."

def show(title, obj):
    print(f"\n[{title}]\n{obj}")



## 3) 텍스트 정규화와 토큰화
- 대소문자 통일, 구두점 제거 여부는 태스크에 맞게 결정
- 토큰화는 언어/도메인별 토크나이저 선택(영어 예시 사용)



In [ ]:
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize

text = sample_text
lowered = text.lower()
# 구두점 제거
punc_table = str.maketrans('', '', string.punctuation)
no_punc = lowered.translate(punc_table)

show("원문", text)
show("소문자", lowered)
show("구두점 제거", no_punc)
show("단어 토큰화", word_tokenize(no_punc))
show("문장 토큰화", sent_tokenize(text))



## 4) 불용어 제거
- 일반 불용어 + 도메인 커스텀 불용어 함께 사용 권장



In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# 필요 리소스 다운로드(최초 1회)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
custom_stopwords = {"it's", "us", "this"}
all_stopwords = stop_words.union(custom_stopwords)

tokens = word_tokenize(no_punc)
filtered = [t for t in tokens if t not in all_stopwords]

lemmatizer = WordNetLemmatizer()
lemmatized = [lemmatizer.lemmatize(t, pos='v') for t in filtered]

show("불용어 제거 전", tokens)
show("불용어 제거 후", filtered)
show("표제어 추출 후", lemmatized)



## 5) 정규표현식 핵심 패턴
- 숫자 제거: `re.sub(r"\d+", "", text)`
- 다중 공백 정리: `re.sub(r"\s+", " ", text).strip()`
- URL/이메일 제거 예시



In [ ]:
sample = "Contact me at test@example.com and visit https://example.com, price 12345."
no_digits = re.sub(r"\d+", "", sample)
no_urls = re.sub(r"https?://\S+", "", no_digits)
no_emails = re.sub(r"\S+@\S+", "", no_urls)
normalized_space = re.sub(r"\s+", " ", no_emails).strip()

show("원문", sample)
show("숫자 제거", no_digits)
show("URL 제거", no_urls)
show("이메일 제거", no_emails)
show("공백 정리", normalized_space)



## 6) 정수 인코딩(Integer Encoding)
- 소규모 예시로 토큰→정수 매핑을 직접 구현
- 실제 프로젝트에선 `Tokenizer`(Keras) 등 사용 가능



In [ ]:
from collections import Counter

# 간단한 코퍼스
docs = [
    "the quick brown fox",
    "jumped over the lazy dog",
    "the fox is quick and the dog is lazy"
]

# 토큰화 및 어휘 집합
tokenized = [d.split() for d in docs]
vocab = Counter([t for doc in tokenized for t in doc])
# 빈도 순 정렬 후 인덱스 부여
vocab_sorted = [w for w, _ in vocab.most_common()]
word2idx = {w: i+1 for i, w in enumerate(vocab_sorted)}  # 0은 패딩 예약

encoded_docs = [[word2idx[t] for t in doc] for doc in tokenized]

show("어휘 크기", len(word2idx))
show("word2idx", word2idx)
show("정수 인코딩 결과", encoded_docs)



## 7) 시퀀스 패딩(Padding)
- 최대 길이 혹은 분위수(예: 95%) 길이로 패딩 길이 결정
- 앞/뒤 패딩 전략은 모델/데이터에 맞게 선택



In [ ]:
def pad_sequences(sequences, maxlen, value=0, padding='post'):
    result = []
    for seq in sequences:
        if len(seq) >= maxlen:
            if padding == 'post':
                result.append(seq[:maxlen])
            else:
                result.append(seq[-maxlen:])
        else:
            if padding == 'post':
                result.append(seq + [value] * (maxlen - len(seq)))
            else:
                result.append([value] * (maxlen - len(seq)) + seq)
    return result

maxlen = 6
padded = pad_sequences(encoded_docs, maxlen=maxlen, value=0, padding='post')
show("패딩 결과", padded)



## 8) 원-핫 인코딩(One-Hot Encoding)
- 분류 레이블 혹은 단어 인덱스를 원-핫 벡터로 표현



In [ ]:
import numpy as np

def to_one_hot(indices, depth):
    arr = np.zeros((len(indices), depth), dtype=int)
    for i, idx in enumerate(indices):
        if 0 <= idx < depth:
            arr[i, idx] = 1
    return arr

sample_indices = [1, 3, 2]
depth = 5
show("원-핫 결과", to_one_hot(sample_indices, depth))



## 9) 워드클라우드(Word Cloud)
- 시각화 전처리: 불용어/소문자/구두점 제거 후 단어 빈도 계산
- 시스템에 한글 폰트 경로가 있는 경우 설정 필요(예시는 영문)



In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text_for_wc = " ".join(lemmatized)
wc = WordCloud(width=600, height=300, background_color='white').generate(text_for_wc)
plt.figure(figsize=(8,4))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()



## 10) 체크리스트 & 실전 팁
- 전처리 정책은 학습/평가/예측에 동일 적용(함수화 권장)
- 불용어는 도메인 맞춤 확장 리스트 유지
- 길이 분포를 보고 패딩 길이 결정(과/미패딩 방지)
- OOV 정책 미리 정의(특수 토큰, 최소 빈도 컷)
- n-gram 범위, 토큰 케이스/구두점 정책은 검증으로 선택



## 11) 복습 퀴즈 & 미니 과제
- 퀴즈
  1) Stemming과 Lemmatization의 차이를 한 문장으로 설명하세요.
  2) OOV를 줄이는 방법 2가지를 적어보세요.
  3) 패딩 길이를 정할 때 확인해야 할 분포는 무엇인가요?
- 미니 과제
  - 같은 말뭉치에 대해 (a) 구두점 제거 유무, (b) 불용어 리스트 변화, (c) n-gram 범위(1, 1-2) 조합을 바꿔 TF-IDF 스코어 상위 단어와 간단 분류 성능의 변화를 비교해보세요.



## 12) 한국어 불용어/형태소 기반 간단 예시
- `KoNLPy`의 `Okt` 형태소기를 사용하여 한국어 토큰화/품사 기반 정규화 예시
- 불용어: 제공된 `ko_stopwords.txt`를 로드하여 적용(도메인에 맞게 확장 권장)
- 주의: Windows 환경에서는 `konlpy` 설치에 자바 런타임이 필요할 수 있음



In [ ]:
# 필요 시 설치: pip install konlpy
from konlpy.tag import Okt

okt = Okt()

korean_text = "이 영화 진짜 재밌다ㅋㅋ 배우 연기가 너무 좋고 연출도 훌륭해요!"

# 형태소 단위 토큰화 + 명사/동사/형용사 위주 추출
tokens_pos = okt.pos(korean_text, norm=True, stem=True)
keep_pos = {"Noun", "Verb", "Adjective"}
filtered_by_pos = [w for w, p in tokens_pos if p in keep_pos]

# 불용어 로드
stop_path = "ko_stopwords.txt"
try:
    with open(stop_path, 'r', encoding='utf-8') as f:
        stop_ko = set([line.strip() for line in f if line.strip()])
except FileNotFoundError:
    stop_ko = set()

filtered_ko = [w for w in filtered_by_pos if w not in stop_ko]

print("원문:", korean_text)
print("품사태깅:", tokens_pos)
print("품사필터:", filtered_by_pos)
print("불용어제거:", filtered_ko)



## 13) TF-IDF 비교 실습
- 전처리 옵션 조합에 따른 TF-IDF 상위 단어 비교
- 옵션 예시: 구두점 제거 유무, 불용어 적용 유무, n-gram=(1) vs (1,2)



In [ ]:
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

corpus = [
    "The Matrix is a groundbreaking sci-fi movie.",
    "This movie has excellent visual effects and a strong plot.",
    "Many consider The Matrix a classic with deep philosophical themes.",
]

# 전처리 함수(옵션)
def preprocess(text, remove_punc=True, to_lower=True, stop=None):
    if to_lower:
        text = text.lower()
    if remove_punc:
        # 모든 구두점 제거를 안전하게 처리
        text = re.sub("[" + re.escape(string.punctuation) + "]", " ", text)
    if stop:
        tokens = text.split()
        text = " ".join([t for t in tokens if t not in stop])
    text = re.sub(r"\s+", " ", text).strip()
    return text

stop_en = set(["the", "a", "and", "is", "this", "with"])

settings = [
    dict(name="A: no_punc, no_stop, uni", remove_punc=False, stop=None, ngram=(1,1)),
    dict(name="B: punc, stop, uni", remove_punc=True, stop=stop_en, ngram=(1,1)),
    dict(name="C: punc, stop, uni+bi", remove_punc=True, stop=stop_en, ngram=(1,2)),
]

rows = []
for s in settings:
    processed = [preprocess(doc, remove_punc=s['remove_punc'], to_lower=True, stop=s['stop']) for doc in corpus]
    vec = TfidfVectorizer(ngram_range=s['ngram'], max_features=15)
    X = vec.fit_transform(processed)
    features = vec.get_feature_names_out()
    scores = X.toarray().sum(axis=0)
    top = sorted(zip(features, scores), key=lambda x: x[1], reverse=True)[:8]
    rows.append({"설정": s['name'], "상위 단어": ", ".join([w for w,_ in top])})

pd.DataFrame(rows)

